The purpose of this fourth experiment is to show that a random network with the same architecture does not give the same explanation.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from random import choices
from minmax.models import vgg
from torchvision.models import vgg as vggn

transform_test = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_reverse = transforms.Compose([
    transforms.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
])

testset = torchvision.datasets.ImageNet(
    root='./files/ImageNet',
    split="val",
    transform=transform_test
)

samples = [8922,13899,16096,4753,9301,37782,35062,25899,23031,44597] + choices(range(len(testset)), k=10) 
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1,
    num_workers=2,
    sampler=samples
)

net, s, r = vgg.vgg19(pretrained=False), 0.2, float(0.1)
net.eval()
net.zero_grad()
net.requires_grad_(False)
netn = vggn.vgg19(pretrained=False)
print("Loading Finished")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from minmax.mm import mmTensor
from minmax.ImageNetClasses import classes
import scipy.ndimage as ndimage
from scipy.stats import spearmanr
from pytorch_grad_cam import GradCAM
import os

OUTPUT_FOLDER = "Experiment_4"
try:
    os.mkdir(OUTPUT_FOLDER)
    print("Output Folder Created")
except FileExistsError:
    print("Output Folder Exists")

def set_scale(m):
    #m.scale_factor = s
    #m.debug_range = True
    #m.debug_relu = True
    m.autoscale_relu = 1/1000

def remove_scale(m):
    if hasattr(m, 'scale_factor'):
        del m.scale_factor
    if hasattr(m, 'debug_range'):
        del m.debug_range
    if hasattr(m, 'debug_relu'):
        del m.debug_relu
    if hasattr(m, 'autoscale_relu'):
        del m.autoscale_relu

expl = {}
for i, (data, target) in enumerate(testloader):
    # Get GradCam Gradient
    inputs = torch.clone(data)
    target_layers = [netn.features[-1]] #VGG
    cam = GradCAM(model=netn, target_layers=target_layers)
    expl_gradcam = cam(input_tensor=inputs, targets=None)[0]
    expl['GradCam'] = cam(input_tensor=inputs, targets=None)[0]
    
    # Get Standard Gradient
    net.apply(remove_scale)
    inputs = torch.clone(data)
    inputs.requires_grad = True
    outputs = net(mmTensor(inputs))

    prediction = outputs.center().argmax().item()
    one_hot = torch.zeros((1,1000))
    one_hot[0,target.item()] = 1

    print("Sample ID: ", samples[i])
    print("True Class: ", classes[target.item()])
    print("Prediction: ", classes[prediction])

    l = torch.sum(outputs.center() * one_hot)
    l.backward()
    
    expl['Gradient'] = inputs.grad.data.detach()[0]
    expl['Gradient'] = np.linalg.norm(expl['Gradient'], axis=0)
    expl['Gradient'] -= expl['Gradient'].min()
    expl['Gradient'] /= expl['Gradient'].max()

    # Get Minmax Gradient
    net.apply(set_scale)
    inputs = mmTensor(data)
    mask = torch.full(data[0].shape, float(r), requires_grad=True)
    inputs.add_mask(mask, mask)
    
    outputs = net(inputs)
    
    l = torch.sum((outputs.upper() - outputs.lower()) * one_hot)
    l.backward()
    
    print("RangeGrad Range: ", outputs.get_range())

    expl['RangeGrad'] = mask.grad.data.detach().numpy()
    expl['RangeGrad'] = np.linalg.norm(expl['RangeGrad'], axis=0)
    expl['RangeGrad'] -= expl['RangeGrad'].min()
    expl['RangeGrad'] /= expl['RangeGrad'].max()
    
    # Get SmoothGrad Gradient
    net.apply(remove_scale)
    expl['SmoothGrad'] = torch.zeros(data[0].shape)
    prop_min = torch.full(outputs.center().shape, +float("inf"))
    prop_max = torch.full(outputs.center().shape, -float("inf"))
    for j in range(35):
        inputs = data + torch.Tensor(np.random.normal(0, 0.15, data[0].shape).astype(np.float32)) 
        inputs.requires_grad = True
        outputs = net(mmTensor(inputs))
        
        prop_min = torch.min(outputs.center(), prop_min)
        prop_max = torch.max(outputs.center(), prop_max)
        
        prediction = outputs.center().argmax().item()
        l = torch.sum(outputs.center() * one_hot)
        l.backward()
        
        expl['SmoothGrad'] += inputs.grad.data.detach()[0]
    
    print("Real Range: ", torch.abs(prop_min - prop_max).mean().item())
    expl['SmoothGrad'] = expl['SmoothGrad'].detach().numpy()
    expl['SmoothGrad'] = np.linalg.norm(expl['SmoothGrad'], axis=0)
    expl['SmoothGrad'] -= expl['SmoothGrad'].min()
    expl['SmoothGrad'] /= expl['SmoothGrad'].max()

    # Draw All
    image = transform_reverse(data)[0].movedim(0,2).numpy()
    plt.clf()
    plt.axis("off")
    plt.imshow(image)
    plt.savefig("{}/{}_Original.png".format(OUTPUT_FOLDER, samples[i]), bbox_inches='tight',pad_inches=0)
    plt.clf()
    
    for n in ["Gradient", "SmoothGrad", "GradCam", "RangeGrad"]:
        try:
            expl[n + "_trained"] = np.load("{}/{}_{}.npy".format("Experiment_1", samples[i], n))
        except FileNotFoundError:
            print("Gradient not found")
            break
        plt.clf()
        plt.axis("off")
        plt.imshow(np.concatenate((image, np.expand_dims(expl[n], axis=2)), axis=2))
        plt.savefig("{}/{}_{}_untrained.png".format(OUTPUT_FOLDER, samples[i], n), bbox_inches='tight',pad_inches=0)
              
        plt.clf()
        plt.axis("off")
        plt.imshow(np.concatenate((image, np.expand_dims(expl[n+"_trained"], axis=2)), axis=2))
        plt.savefig("{}/{}_{}_trained.png".format(OUTPUT_FOLDER, samples[i], n), bbox_inches='tight',pad_inches=0)

    else:
        plt.clf()

        fig, axs = plt.subplots(2,5)
        for i in axs:
            for j in i:
                j.axis("off")
        fig.set_figwidth(20)
        fig.set_figheight(10)

        axs[0][0].set_title("Original")
        axs[0][0].imshow(image)

        axs[0][1].set_title("Gradient")
        axs[0][1].imshow(np.concatenate((image, np.expand_dims(expl['Gradient'], axis=2)), axis=2))
        axs[1][1].imshow(np.concatenate((image, np.expand_dims(expl['Gradient_trained'], axis=2)), axis=2))

        axs[0][2].set_title("SmoothGrad")
        axs[0][2].imshow(np.concatenate((image, np.expand_dims(expl['SmoothGrad'], axis=2)), axis=2))
        axs[1][2].imshow(np.concatenate((image, np.expand_dims(expl['SmoothGrad_trained'], axis=2)), axis=2))

        axs[0][3].set_title("GradCam")
        axs[0][3].imshow(np.concatenate((image, np.expand_dims(expl['GradCam'], axis=2)), axis=2))
        axs[1][3].imshow(np.concatenate((image, np.expand_dims(expl['GradCam_trained'], axis=2)), axis=2))

        axs[0][4].set_title("RangeGrad")
        axs[0][4].imshow(np.concatenate((image, np.expand_dims(expl['RangeGrad'], axis=2)), axis=2))
        axs[1][4].imshow(np.concatenate((image, np.expand_dims(expl['RangeGrad_trained'], axis=2)), axis=2))
        plt.show()